## 1. Contagem de logins em telemóveis Android
Foi obtido um conjunto de registos a partir de logins em diferentes dispositivos. Uma amostra do dataset activity.csv é apresentada em seguida:

In [197]:
import pandas as pd

from matplotlib import pyplot as plt

df = pd.read_csv('../data/activity.csv')
df.head()

,id,user_id,created_at,device
0,1,101,2015-02-08,windows
1,2,103,2015-01-07,mac
2,3,132,2014-12-09,mac
3,4,247,2015-07-13,android
4,5,313,2015-06-20,android


In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1200 non-null   int64 
 1   user_id     1200 non-null   int64 
 2   created_at  1200 non-null   object
 3   device      1200 non-null   object
dtypes: int64(2), object(2)
memory usage: 37.6+ KB


In [199]:
df.created_at = pd.to_datetime(df.created_at)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          1200 non-null   int64         
 1   user_id     1200 non-null   int64         
 2   created_at  1200 non-null   datetime64[ns]
 3   device      1200 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 37.6+ KB


Apresente o número de entradas no dispositivo Android por mês.

In [200]:
import plotly.express as px

#plot logins mensais no device android
df_android = df[df.device == 'android']
df_android


,id,user_id,created_at,device
3,4,247,2015-07-13,android
4,5,313,2015-06-20,android
5,6,317,2014-08-08,android
8,9,443,2015-06-17,android
10,11,506,2015-04-06,android
...,...,...,...,...
1190,1191,54666,2014-08-09,android
1191,1192,54703,2015-01-29,android
1195,1196,54933,2015-04-04,android
1196,1197,54937,2014-08-05,android


In [201]:
#agregar por ano e mes
df_android['year_month'] = df_android.created_at.dt.strftime('%Y-%m')

C:\Users\TG\AppData\Local\Temp\ipykernel_18268\1719461271.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [202]:
# drop id, user_id e device
df_android = df_android.drop(['id', 'user_id', 'device'], axis=1)
df_android

,created_at,year_month
3,2015-07-13,2015-07
4,2015-06-20,2015-06
5,2014-08-08,2014-08
8,2015-06-17,2015-06
10,2015-04-06,2015-04
...,...,...
1190,2014-08-09,2014-08
1191,2015-01-29,2015-01
1195,2015-04-04,2015-04
1196,2014-08-05,2014-08


In [203]:
# contando o numero de logins por mes
df_android = df_android.groupby('year_month').count()
df_android

,created_at
year_month,
2014-07,5
2014-08,22
2014-09,27
2014-10,27
2014-11,25
2014-12,23
2015-01,14
2015-02,26
2015-03,31


In [204]:
# plotando logins mensais no device android usando plotly

px.bar(df_android, y='created_at', title='Logins mensais no device android', labels={'created_at': 'número de logins', 'year_month': 'mês'})

## 2. Benfica vs Porto
A época desportiva 2020/21 foi extremamente competitiva e com várias reviravoltas. Dado o número de pontos do Benfica e Porto, jornada a jornada, pretende-se representar graficamente as pontuações e lideranças.

Um excerto com as pontuações do Benfica e Porto, jornada a jornada, é apresentado em seguida

In [205]:
df_times = pd.read_csv('../data/benfica_porto.csv', delim_whitespace=True)
df_times.head(5)

,benfica,porto
0,3,3
1,6,6
2,7,9
3,10,12
4,13,15


In [206]:
df_times.columns

Index(['benfica', 'porto'], dtype='object')

In [207]:
# criando coluna cor azul se o porto estiver na frente, vermelho se benfica na frente e amarelo se empate de acordo com a liderança que depende de quem tem mais pontos por linha
df_times['cor'] = df_times.apply(lambda x: 'blue' if x['porto'] > x['benfica'] else 'red' if x['benfica'] > x['porto'] else 'yellow', axis=1)
df_times

,benfica,porto,cor
0,3,3,yellow
1,6,6,yellow
2,7,9,blue
3,10,12,blue
4,13,15,blue
5,14,18,blue
6,17,21,blue
7,20,24,blue
8,23,27,blue
9,26,30,blue


In [208]:
# importando graphic object do plotly
import plotly.graph_objects as go

# plot histograma de lideranças
fig = go.Figure(data=[  go.Bar(y=df_times['benfica'], 
                             x=df_times.index, name='benfica',
                             marker=dict(color= 'red')),
                        go.Bar(y=df_times['porto'], 
                             x=df_times.index, name='porto',
                             marker=dict(color= 'blue')),
                         go.Scatter(x = df_times.index,
                                    y = df_times['benfica'] + 10,
                                    marker=dict(color= df_times['cor']),
                                    name='diferença',
                                    mode='markers',
                                    showlegend=False)])

fig.update_layout(title='Benfica vs Porto', xaxis_title='Jornadas', yaxis_title='Pontos')

fig.show()